In [14]:
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [15]:
import ct.sm as sm
import cantera as ct
import scipy.constants as C
import matplotlib.pyplot as plt
# import CoolProp.CoolProp as cp
import importlib
# import sm
import numpy as np
# import pint
importlib.reload(sm)

<module 'ct.sm' from 'g:\\Autism\\ss13\\supermattercore\\supermattercore\\ct\\sm.py'>

In [142]:
ct_R = ct.constants.gas_constant/1e3
ct_R

8.31446261815324

In [16]:
s = ct.Solution('./ct/xgm.yaml')
ct_R = ct.constants.gas_constant/1e3

nu_N2 = sm.CANISTER_VOLUME*sm.ENG_CANISTER_PRESSURE/ct_R/293
"""Amount of substance in a single engine N2 canister."""

s.TPX = 293, sm.ENG_CANISTER_PRESSURE, 'N2: 1'
q = ct.Quantity(s, moles=nu_N2/1e3)
assert q.V == sm.CANISTER_VOLUME

In [143]:
s()


  xgm:

       temperature   270.02 K
          pressure   5.1841e+05 Pa
           density   6.4688 kg/m^3
  mean mol. weight   28.014 kg/kmol
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy            -28435       -7.9657e+05  J
   internal energy       -1.0857e+05       -3.0416e+06  J
           entropy           -584.67            -16379  J/K
    Gibbs function        1.2944e+05         3.626e+06  J
 heat capacity c_p            1010.7             28314  J/K
 heat capacity c_v            713.91             20000  J/K

                      mass frac. Y      mole frac. X     chem. pot. / RT
                     ---------------   ---------------   ---------------
                N2                 1                 1            1.6151
     [   +3 minor]                 0                 0  



In [38]:
p = sm.PipeNet(sm.CANISTER_VOLUME, q)
# vOldHot = .2+2.345+.7+37.5+.2+.315+.6+2
vNewCold = 1.47 + 13.37 + 0.77 + 3*.2
pLoop = sm.PipeNet(vNewCold, None)
# 1.96+1.12+7.35+1.19+.4

In [39]:
p.grow(0.2)
p.qty.V, p.qty.volume, p.Nu

(1.2, 1.2, 3.7433231185098275)

In [40]:
pmp = sm.Pump()
he = sm.HeatExchanger(191,0)

In [140]:
pmp.process(p, pLoop)
pLoop.qty, *_ = he.cool_iter(pLoop.qty)
pLoop.qty, *_ = he.cool_iter(pLoop.qty)
pmp.process(p, pLoop)
pLoop.qty, *_ = he.cool_iter(pLoop.qty)
pLoop.qty, *_ = he.cool_iter(pLoop.qty)
pLoop.P/1e5

[ct.sm.Pump] Source entropy 231.83 J/mol/K -> Sink entropy 176.79 J/mol/K
[ct.sm.Pump] Specific entropy change -55.042 J/mol/K
[ct.sm.Pump] Specific power 14883.4 W/mol
[ct.sm.Pump] Transferred 0.059628 mols
[ct.sm.Pump] Source entropy 233.35 J/mol/K -> Sink entropy 176.79 J/mol/K
[ct.sm.Pump] Specific entropy change -56.555 J/mol/K
[ct.sm.Pump] Specific power 15281.7 W/mol
[ct.sm.Pump] Transferred 0.04969 mols


5.184057888438605

In [141]:
pLoop.T

270.01642414383286

In [120]:
for i in range(10):
    pLoop.qty, *_ = he.cool_iter(pLoop.qty)
# q.T,q.P
pLoop.T

276.6759784424249

In [110]:
pLoop.P

354580.5717895382